In [1]:
import pandas as pd

# Step 1: Define path
excel_path = '/Users/emilsafarov/Library/CloudStorage/OneDrive-Personal/CF/A6/1_sourceing_open_data/US project/Data/Output/essential_dataset_v1.xlsx'
# Step 2: Load all sheets into a dictionary
all_sheets = pd.read_excel(excel_path, sheet_name=None)


In [2]:
target_sheets_1 = [
    "air_pollution", "children_lunch", "children_poverty", "dentists_raw_value",
    "disconnected_youth", "drinking_water", "drive_alone", "drug_overdose_deaths",
    "exercise_access", "firearm_fatalities", "flu_vaccinations_raw_value", "food_environment_index",
    "food_insecurity", "frequent_physical_distress", "high_housing_cost", "high_school_graduation",
    "hiv_prevalence_raw_value", "homeownership", "homicides", "housing_facilities",
    "income_inequality", "injury_deaths", "insufficient_sleep_raw_value", "juvenile_arrests",
    "life_expectancy", "limited_access_healthy_foods", "long_commute", "mammography_screening",
    "math_scores", "median_income", "mental_health_providers_raw_val", "motor_vehicle_crash_deaths",
    "not_proficient_english", "poor_or_fair_health", "poor_physical_health", "population_raw_value",
    "primary_care_physicians", "reading_scores", "severe_housing", "severe_housing_problems",
    "some_college", "suicides_raw_value", "traffic_volume", "unemployment",
    "uninsured_raw_value", "violent_crime"
]


In [3]:
# 🔁 Loop through target sheets and check for duplicate FIPS codes
for sheet in target_sheets_1:
    df = all_sheets[sheet].copy()
    
    if 'FIPS' in df.columns:
        # Standardize FIPS to 5-digit strings
        df['FIPS'] = df['FIPS'].astype(str).str.zfill(5)
        
        # Count duplicates
        duplicates = df['FIPS'].duplicated().sum()
        
        # Report if any found
        if duplicates > 0:
            print(f"❗ Sheet '{sheet}' has {duplicates} duplicate FIPS")
    else:
        print(f"⚠️ Sheet '{sheet}' has no 'FIPS' column")


❗ Sheet 'air_pollution' has 2 duplicate FIPS
❗ Sheet 'children_lunch' has 1 duplicate FIPS
❗ Sheet 'children_poverty' has 1 duplicate FIPS
❗ Sheet 'dentists_raw_value' has 1 duplicate FIPS
❗ Sheet 'disconnected_youth' has 1 duplicate FIPS
❗ Sheet 'drinking_water' has 2 duplicate FIPS
❗ Sheet 'drive_alone' has 1 duplicate FIPS
❗ Sheet 'drug_overdose_deaths' has 1 duplicate FIPS
❗ Sheet 'exercise_access' has 1 duplicate FIPS
❗ Sheet 'firearm_fatalities' has 1 duplicate FIPS
❗ Sheet 'flu_vaccinations_raw_value' has 1 duplicate FIPS
❗ Sheet 'food_environment_index' has 1 duplicate FIPS
❗ Sheet 'food_insecurity' has 1 duplicate FIPS
❗ Sheet 'frequent_physical_distress' has 1 duplicate FIPS
❗ Sheet 'high_housing_cost' has 1 duplicate FIPS
❗ Sheet 'high_school_graduation' has 1 duplicate FIPS
❗ Sheet 'hiv_prevalence_raw_value' has 1 duplicate FIPS
❗ Sheet 'homeownership' has 1 duplicate FIPS
❗ Sheet 'homicides' has 1 duplicate FIPS
❗ Sheet 'housing_facilities' has 1 duplicate FIPS
❗ Sheet 'in

In [4]:
# Initialize merged DataFrame
merged_df_1 = None

# Merge all sheets side-by-side on FIPS
for sheet_name in target_sheets_1:
    df = all_sheets[sheet_name].copy()

    # Standardize FIPS
    df['FIPS'] = df['FIPS'].astype(str).str.zfill(5)

    # Keep only one row per FIPS, State, County (take mean of numeric values)
    group_keys = ['FIPS']
    if 'State' in df.columns and 'County' in df.columns:
        group_keys = ['FIPS', 'State', 'County']

    df = df.groupby(group_keys, as_index=False).mean(numeric_only=True)

    # Rename value columns (but keep FIPS, State, County intact)
    df = df.rename(columns={col: f"{sheet_name}__{col}" for col in df.columns if col not in group_keys})

    # Merge
    if merged_df_1 is None:
        merged_df_1 = df
    else:
        merged_df_1 = pd.merge(merged_df_1, df, on=group_keys, how="outer")

# ✅ Final merged_df_1 contains FIPS, State, County, and all indicators
print("✅ Final merged shape:", merged_df_1.shape)
print(merged_df_1.head())


✅ Final merged shape: (3198, 278)
    FIPS State          County  air_pollution__air_pollution_2020  \
0  00000    US   United States                                8.6   
1  01000    AL         Alabama                               11.0   
2  01001    AL  Autauga County                               11.7   
3  01003    AL  Baldwin County                               10.3   
4  01005    AL  Barbour County                               11.5   

   air_pollution__air_pollution_2021  air_pollution__air_pollution_2022  \
0                                7.2                                7.5   
1                                9.2                                9.0   
2                               10.4                                9.5   
3                                7.2                                7.2   
4                                9.4                                9.0   

   air_pollution__air_pollution_2023  air_pollution__air_pollution_2024  \
0                        

In [5]:
merged_df_1

,FIPS,State,County,air_pollution__air_pollution_2020,air_pollution__air_pollution_2021,air_pollution__air_pollution_2022,air_pollution__air_pollution_2023,air_pollution__air_pollution_2024,air_pollution__air_pollution_mean,children_lunch__children_lunch_2020,...,uninsured_raw_value__uninsured_raw_value_2022,uninsured_raw_value__uninsured_raw_value_2023,uninsured_raw_value__uninsured_raw_value_2024,uninsured_raw_value__uninsured_mean,violent_crime__violent_crime_2020,violent_crime__violent_crime_2021,violent_crime__violent_crime_2022,violent_crime__violent_crime_2023,violent_crime__violent_crime_2024,violent_crime__violent_crime_mean
0,00000,US,United States,8.6,7.2,7.5,7.4,7.35,7.61,0.522823,...,0.108405,0.104450,0.102285,0.104333,386.464896,386.464896,386.464896,NaN,NaN,386.464896
1,01000,AL,Alabama,11.0,9.2,9.0,9.3,9.30,9.56,0.558742,...,0.115995,0.118227,0.116934,0.116090,479.919182,479.919182,479.919182,NaN,NaN,479.919182
2,01001,AL,Autauga County,11.7,10.4,9.5,10.0,10.00,10.32,0.434087,...,0.093512,0.105594,0.100186,0.097396,272.282220,272.282220,272.282220,NaN,NaN,272.282220
3,01003,AL,Baldwin County,10.3,7.2,7.2,7.6,7.60,7.98,0.484866,...,0.109068,0.108749,0.109971,0.114562,203.660396,203.660396,203.660396,NaN,NaN,203.660396
4,01005,AL,Barbour County,11.5,9.4,9.0,9.4,9.40,9.74,0.632923,...,0.129915,0.143683,0.126776,0.131568,414.277861,414.277861,414.277861,NaN,NaN,414.277861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3193,56039,WY,Teton County,4.9,4.5,5.2,3.6,3.60,4.36,0.222070,...,0.174560,0.145076,0.162863,0.155302,178.429982,178.429982,178.429982,NaN,NaN,178.429982
3194,56041,WY,Uinta County,5.9,6.2,6.9,5.3,5.30,5.92,0.377846,...,0.140600,0.138928,0.144315,0.135775,71.006517,71.006517,71.006517,NaN,NaN,71.006517
3195,56043,WY,Washakie County,4.8,4.1,5.2,3.5,3.50,4.22,0.375087,...,0.158514,0.158872,0.181106,0.162828,78.266105,78.266105,78.266105,NaN,NaN,78.266105
3196,56045,WY,Weston County,4.1,3.0,4.3,3.2,3.20,3.56,0.306548,...,0.143161,0.130758,0.167536,0.140458,157.128680,157.128680,157.128680,NaN,NaN,157.128680


In [6]:
# Load the two population-related sheets
pop_df_1 = all_sheets["Population"].copy()
pop_df_2 = all_sheets["CO-EST2024-POP"].copy()

# Standardize column names
pop_df_1.columns = pop_df_1.columns.str.strip()
pop_df_2.columns = pop_df_2.columns.str.strip()

# Merge on State and County
merged_df_2 = pd.merge(pop_df_1, pop_df_2, on=["State", "County"], how="outer", suffixes=("_pop", "_est2024"))

# ✅ Preview result
print("✅ Merged shape:", merged_df_2.shape)
print(merged_df_2.head())


✅ Merged shape: (3144, 21)
  State                      County  Total Population  \
0    AK      Aleutians East Borough              3423   
1    AK  Aleutians West Census Area              5178   
2    AK      Anchorage Municipality            289069   
3    AK          Bethel Census Area             18487   
4    AK         Bristol Bay Borough               878   

   Young Children (0–4 years)  Children (5–14 years)  \
0                         106                    215   
1                         119                    380   
2                       18776                  38742   
3                        1957                   3506   
4                          64                     64   

   Adolescents (15–19 years)  Young Adults (20–24 years)  \
0                        141                         354   
1                        382                         376   
2                      17885                       21171   
3                       1538                        1

In [7]:
merged_df_2

,State,County,Total Population,Young Children (0–4 years),Children (5–14 years),Adolescents (15–19 years),Young Adults (20–24 years),Emerging Workforce (25–34 years),Prime Working Age (35–54 years),Pre-Retirement (55–64 years),...,Diversity Index,Hispanic or Latino Population,Non-Hispanic or Latino Population,Hispanic or Latino Percentage,Non-Hispanic or Latino Percentage,pop_2020,pop_2021,pop_2022,pop_2023,pop_2024
0,AK,Aleutians East Borough,3423,106,215,141,354,516,1158,540,...,0.772724,360,3063,10.517090,89.482910,3426,3416,3453,3529,3632
1,AK,Aleutians West Census Area,5178,119,380,382,376,964,1897,730,...,0.742459,715,4463,13.808420,86.191580,5223,5156,5115,5257,5424
2,AK,Anchorage Municipality,289069,18776,38742,17885,21171,48210,74675,33674,...,0.623138,26953,262116,9.324071,90.675929,290905,288921,287520,287932,289600
3,AK,Bethel Census Area,18487,1957,3506,1538,1305,3044,3630,2025,...,0.280340,257,18230,1.390166,98.609834,18670,18589,18272,18199,18237
4,AK,Bristol Bay Borough,878,64,64,22,32,213,186,176,...,0.678699,73,805,8.314351,91.685649,830,842,877,859,884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,WY,Sweetwater County,41786,2361,6272,2949,2529,5263,11348,5162,...,0.308875,6924,34862,16.570143,83.429857,42196,41630,41322,41288,41273
3140,WY,Teton County,23358,959,2433,980,1031,4051,6974,2843,...,0.336397,3445,19913,14.748694,85.251306,23384,23613,23341,23368,23272
3141,WY,Uinta County,20605,1269,3361,1454,1134,2426,5078,2460,...,0.168213,2042,18563,9.910216,90.089784,20461,20690,20722,20724,20621
3142,WY,Washakie County,7708,321,903,648,335,734,1847,1104,...,0.298463,1087,6621,14.102231,85.897769,7663,7727,7728,7736,7662


In [8]:
# Extract unique FIPS–State–County mapping
fips_lookup_df = merged_df_1[['FIPS', 'State', 'County']].drop_duplicates()


In [9]:
# Add FIPS to merged_df_2 using State + County as the key
merged_df_2 = pd.merge(merged_df_2, fips_lookup_df, on=["State", "County"], how="left")

# Confirm FIPS added
print("✅ FIPS added to merged_df_2")
print(merged_df_2[['FIPS', 'State', 'County']].head())


✅ FIPS added to merged_df_2
    FIPS State                      County
0  02013    AK      Aleutians East Borough
1  02016    AK  Aleutians West Census Area
2    NaN    AK      Anchorage Municipality
3  02050    AK          Bethel Census Area
4  02060    AK         Bristol Bay Borough


In [10]:
# 🔍 Filter rows where FIPS was not found (NaN after merge)
missing_fips_rows = merged_df_2[merged_df_2['FIPS'].isna()]

# 📊 Show how many are missing
print(f"❗ Rows with missing FIPS: {missing_fips_rows.shape[0]}")

# 👀 Preview them
print(missing_fips_rows[['State', 'County']])


❗ Rows with missing FIPS: 22
     State                                          County
2       AK                          Anchorage Municipality
12      AK                         Juneau City and Borough
22      AK                              Petersburg Borough
24      AK                          Sitka City and Borough
27      AK                       Wrangell City and Borough
28      AK                        Yakutat City and Borough
309     CT                         Capitol Planning Region
310     CT              Greater Bridgeport Planning Region
311     CT  Lower Connecticut River Valley Planning Region
312     CT                Naugatuck Valley Planning Region
313     CT        Northeastern Connecticut Planning Region
314     CT                 Northwest Hills Planning Region
315     CT       South Central Connecticut Planning Region
316     CT        Southeastern Connecticut Planning Region
317     CT             Western Connecticut Planning Region
745     IL                 

In [11]:
fmr_df = all_sheets["fmr"].copy()

# Make sure FIPS is standardized
fmr_df['FIPS'] = fmr_df['FIPS'].astype(str).str.zfill(5)

# (Optional) Check available columns
# print(fmr_df.columns)


In [12]:
# Extract State + County from missing rows
missing_fips_rows = merged_df_2[merged_df_2['FIPS'].isna()][['State', 'County']].drop_duplicates()

# Check if these rows exist in merged_df_1
merged_1_check = pd.merge(missing_fips_rows, merged_df_1[['FIPS', 'State', 'County']], on=['State', 'County'], how='left')
print("🔎 FIPS from merged_df_1:")
print(merged_1_check)

# Check if they exist in fmr_df
fmr_check = pd.merge(missing_fips_rows, fmr_df[['FIPS', 'State', 'County']], on=['State', 'County'], how='left')
print("\n🔎 FIPS from fmr_df:")
print(fmr_check)


🔎 FIPS from merged_df_1:
   State                                          County FIPS
0     AK                          Anchorage Municipality  NaN
1     AK                         Juneau City and Borough  NaN
2     AK                              Petersburg Borough  NaN
3     AK                          Sitka City and Borough  NaN
4     AK                       Wrangell City and Borough  NaN
5     AK                        Yakutat City and Borough  NaN
6     CT                         Capitol Planning Region  NaN
7     CT              Greater Bridgeport Planning Region  NaN
8     CT  Lower Connecticut River Valley Planning Region  NaN
9     CT                Naugatuck Valley Planning Region  NaN
10    CT        Northeastern Connecticut Planning Region  NaN
11    CT                 Northwest Hills Planning Region  NaN
12    CT       South Central Connecticut Planning Region  NaN
13    CT        Southeastern Connecticut Planning Region  NaN
14    CT             Western Connecticut Plan

In [13]:
# Dictionary of manually fixed FIPS codes
manual_fips_fixes = {
    ('AK', 'Anchorage Municipality'): '02020',
    ('AK', 'Juneau City and Borough'): '02110',
    ('AK', 'Petersburg Borough'): '02195',
    ('AK', 'Sitka City and Borough'): '02220',
    ('AK', 'Wrangell City and Borough'): '02275',
    ('AK', 'Yakutat City and Borough'): '02282',
    
    ('CT', 'Capitol Planning Region'): '00001',
    ('CT', 'Greater Bridgeport Planning Region'): '00002',
    ('CT', 'Lower Connecticut River Valley Planning Region'): '00003',
    ('CT', 'Naugatuck Valley Planning Region'): '00004',
    ('CT', 'Northeastern Connecticut Planning Region'): '00005',
    ('CT', 'Northwest Hills Planning Region'): '00006',
    ('CT', 'South Central Connecticut Planning Region'): '00007',
    ('CT', 'Southeastern Connecticut Planning Region'): '00008',
    ('CT', 'Western Connecticut Planning Region'): '00009',

    ('IL', 'LaSalle County'): '17099',
    ('IN', 'DeKalb County'): '18033',
    ('IN', 'LaGrange County'): '18087',
    ('IN', 'LaPorte County'): '18091',
    ('NM', 'De Baca County'): '35011',
    ('NM', 'Doña Ana County'): '35013',
    ('PA', 'McKean County'): '42083'
}

# Apply manual corrections
merged_df_2['FIPS'] = merged_df_2.apply(
    lambda row: manual_fips_fixes.get((row['State'], row['County']), row['FIPS']),
    axis=1
)


In [14]:
merged_df_2

,State,County,Total Population,Young Children (0–4 years),Children (5–14 years),Adolescents (15–19 years),Young Adults (20–24 years),Emerging Workforce (25–34 years),Prime Working Age (35–54 years),Pre-Retirement (55–64 years),...,Hispanic or Latino Population,Non-Hispanic or Latino Population,Hispanic or Latino Percentage,Non-Hispanic or Latino Percentage,pop_2020,pop_2021,pop_2022,pop_2023,pop_2024,FIPS
0,AK,Aleutians East Borough,3423,106,215,141,354,516,1158,540,...,360,3063,10.517090,89.482910,3426,3416,3453,3529,3632,02013
1,AK,Aleutians West Census Area,5178,119,380,382,376,964,1897,730,...,715,4463,13.808420,86.191580,5223,5156,5115,5257,5424,02016
2,AK,Anchorage Municipality,289069,18776,38742,17885,21171,48210,74675,33674,...,26953,262116,9.324071,90.675929,290905,288921,287520,287932,289600,02020
3,AK,Bethel Census Area,18487,1957,3506,1538,1305,3044,3630,2025,...,257,18230,1.390166,98.609834,18670,18589,18272,18199,18237,02050
4,AK,Bristol Bay Borough,878,64,64,22,32,213,186,176,...,73,805,8.314351,91.685649,830,842,877,859,884,02060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,WY,Sweetwater County,41786,2361,6272,2949,2529,5263,11348,5162,...,6924,34862,16.570143,83.429857,42196,41630,41322,41288,41273,56037
3141,WY,Teton County,23358,959,2433,980,1031,4051,6974,2843,...,3445,19913,14.748694,85.251306,23384,23613,23341,23368,23272,56039
3142,WY,Uinta County,20605,1269,3361,1454,1134,2426,5078,2460,...,2042,18563,9.910216,90.089784,20461,20690,20722,20724,20621,56041
3143,WY,Washakie County,7708,321,903,648,335,734,1847,1104,...,1087,6621,14.102231,85.897769,7663,7727,7728,7736,7662,56043


In [15]:
# 🔍 Filter rows where FIPS, State, or County is missing
missing_key_info = merged_df_2[
    merged_df_2[['FIPS', 'State', 'County']].isna().any(axis=1)
]

# 👀 Display them
print("❗ Rows with missing FIPS, State, or County:")
print(missing_key_info[['FIPS', 'State', 'County']])


❗ Rows with missing FIPS, State, or County:
Empty DataFrame
Columns: [FIPS, State, County]
Index: []


In [16]:
# 🔍 Count how many FIPS in merged_df_1 also exist in merged_df_2
# This tells you how many rows will successfully match during the merge
print("🔍 FIPS match in merged_df_2:", merged_df_1['FIPS'].isin(merged_df_2['FIPS']).sum())

# 🔍 Count how many FIPS in merged_df_1 also exist in fmr_df
# Useful to ensure FMR data can be merged without losing rows
print("🔍 FIPS match in fmr_df:", merged_df_1['FIPS'].isin(fmr_df['FIPS']).sum())


🔍 FIPS match in merged_df_2: 3137
🔍 FIPS match in fmr_df: 3144


In [17]:
print("📦 merged_df_1 (Main dataset) rows:", merged_df_1.shape[0])
print("📦 merged_df_2 (Population data) rows:", merged_df_2.shape[0])
print("📦 fmr_df (Fair Market Rent) rows:", fmr_df.shape[0])


📦 merged_df_1 (Main dataset) rows: 3198
📦 merged_df_2 (Population data) rows: 3145
📦 fmr_df (Fair Market Rent) rows: 3251


In [18]:
# 🔍 Find duplicated rows based on FIPS, State, and County
fmr_duplicates = fmr_df[fmr_df.duplicated(subset=['FIPS', 'State', 'County'], keep=False)]

# 📊 Show how many duplicated entries
print(f"❗ Duplicated rows in fmr_df: {fmr_duplicates.shape[0]}")

# 👀 Display the duplicated rows
print(fmr_duplicates.sort_values(by=['FIPS', 'State', 'County']))


❗ Duplicated rows in fmr_df: 0
Empty DataFrame
Columns: [FIPS, State, County, fmr_0_2021, fmr_1_2021, fmr_2_2021, fmr_3_2021, fmr_4_2021, fmr_0_2020, fmr_1_2020, fmr_2_2020, fmr_3_2020, fmr_4_2020, fmr_0_2023, fmr_1_2023, fmr_2_2023, fmr_3_2023, fmr_4_2023, fmr_0_2022, fmr_1_2022, fmr_2_2022, fmr_3_2022, fmr_4_2022, fmr_0_2024, fmr_1_2024, fmr_2_2024, fmr_3_2024, fmr_4_2024]
Index: []

[0 rows x 28 columns]


In [19]:
# 🔍 Find rows with duplicated FIPS only
fmr_dup_fips = fmr_df[fmr_df.duplicated(subset='FIPS', keep=False)]

# 📊 Show how many FIPS codes are duplicated
print(f"❗ Rows with duplicated FIPS in fmr_df: {fmr_dup_fips.shape[0]}")

# 👀 Display the duplicated rows sorted by FIPS
print(fmr_dup_fips.sort_values(by='FIPS'))


❗ Rows with duplicated FIPS in fmr_df: 159
       FIPS State             County  fmr_0_2021  fmr_1_2021  fmr_2_2021  \
3236  09001    CT   Fairfield County      1138.0      1350.0      1725.0   
3235  09001    CO        Yuma County       629.0       634.0       835.0   
3238  09003    CT    Hartford County       889.0      1091.0      1347.0   
3237  09003    CT   Fairfield County      1292.0      1621.0      1958.0   
3240  09005    CT  Litchfield County       806.0       950.0      1222.0   
...     ...   ...                ...         ...         ...         ...   
2700  50023    VT     Rutland County       764.0       779.0       918.0   
2703  50025    VT     Windham County       697.0       798.0       993.0   
2702  50025    VT  Washington County       717.0       824.0      1086.0   
2704  50027    VT     Windham County       697.0       798.0       993.0   
2705  50027    VT     Windsor County       715.0       837.0      1007.0   

      fmr_3_2021  fmr_4_2021  fmr_0_2020  fm

In [20]:
import pandas as pd

# Step 1: Create FIPS lookup from merged_df_1 and rename FIPS
fips_lookup_df = merged_df_1[['State', 'County', 'FIPS']].drop_duplicates()
fips_lookup_df = fips_lookup_df.rename(columns={'FIPS': 'FIPS_corrected'})

# Step 2: Merge into fmr_df using State + County
fmr_df = pd.merge(
    fmr_df,
    fips_lookup_df,
    on=['State', 'County'],
    how='left'  # Keep all rows from fmr_df
)

# Step 3: Show unmatched rows (where merge failed = no FIPS_corrected found)
mismatched_names = fmr_df[fmr_df['FIPS_corrected'].isna()]

print(f"❗ Rows with unmatched State + County: {mismatched_names.shape[0]}")
print(mismatched_names[['State', 'County', 'FIPS']])

# Optional: Show rows where FIPS exists but differs from corrected one
fips_conflicts = fmr_df[
    (fmr_df['FIPS_corrected'].notna()) &
    (fmr_df['FIPS'] != fmr_df['FIPS_corrected'])
]

print(f"\n🔁 Rows with differing FIPS values: {fips_conflicts.shape[0]}")
print(fips_conflicts[['State', 'County', 'FIPS', 'FIPS_corrected']])


❗ Rows with unmatched State + County: 12
     State                     County   FIPS
466     IN              DeKalb County  18035
493     IN            LaGrange County  18089
495     IN             LaPorte County  18093
742     AK     Anchorage Municipality  02020
807     AK    Juneau City and Borough  02110
887     AK         Petersburg Borough  02195
953     AK     Sitka City and Borough  02220
957     AK  Wrangell City and Borough  02282
958     AK   Yakutat City and Borough  02290
1338    MO              Sullivan part  29077
1648    NM             De Baca County  35031
2147    PA              McKean County  42105

🔁 Rows with differing FIPS values: 3092
     State                County   FIPS FIPS_corrected
0       CT        Windham County  10001          09015
1       DE           Kent County  10003          10001
2       DE     New Castle County  10005          10003
53      DE         Sussex County  11001          10005
71      DC  District of Columbia  12001          11000
...

In [21]:
fmr_df

,FIPS,State,County,fmr_0_2021,fmr_1_2021,fmr_2_2021,fmr_3_2021,fmr_4_2021,fmr_0_2020,fmr_1_2020,...,fmr_1_2022,fmr_2_2022,fmr_3_2022,fmr_4_2022,fmr_0_2024,fmr_1_2024,fmr_2_2024,fmr_3_2024,fmr_4_2024,FIPS_corrected
0,10001,CT,Windham County,816.0,818.0,1036.0,1307.0,1695.0,787.0,811.0,...,854.0,1110.0,1388.0,1875.0,1186.0,1194.0,1569.0,1921.0,2380.0,09015
1,10003,DE,Kent County,878.0,908.0,1066.0,1526.0,1785.0,880.0,893.0,...,856.0,1002.0,1423.0,1647.0,1110.0,1117.0,1368.0,1928.0,2152.0,10001
2,10005,DE,New Castle County,900.0,1040.0,1260.0,1567.0,1796.0,864.0,1013.0,...,1071.0,1298.0,1605.0,1837.0,1303.0,1451.0,1737.0,2100.0,2422.0,10003
3,01001,AL,Autauga County,640.0,766.0,908.0,1148.0,1520.0,583.0,702.0,...,764.0,914.0,1156.0,1494.0,836.0,913.0,1092.0,1383.0,1753.0,01001
4,01003,AL,Baldwin County,718.0,723.0,922.0,1249.0,1584.0,744.0,749.0,...,777.0,1017.0,1348.0,1715.0,1051.0,1056.0,1362.0,1670.0,2114.0,01003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247,09011,CT,New London County,828.0,991.5,1254.5,1709.5,2123.5,826.5,953.5,...,1020.5,1286.0,1742.5,2179.0,1102.5,1277.0,1626.0,2134.5,2740.5,09011
3248,09013,CT,New London County,810.0,976.0,1227.0,1584.0,2027.0,815.0,938.0,...,1006.0,1254.0,1616.0,2102.0,1055.0,1263.0,1556.0,2030.0,2625.0,09011
3249,09013,CT,Tolland County,889.0,1091.0,1347.0,1675.0,1958.0,801.0,993.0,...,1054.0,1302.0,1609.0,1907.0,1121.0,1319.0,1654.0,2000.0,2425.0,09013
3250,09015,CT,Tolland County,889.0,1091.0,1347.0,1675.0,1958.0,801.0,993.0,...,1054.0,1302.0,1609.0,1907.0,1121.0,1319.0,1654.0,2000.0,2425.0,09013


In [22]:
# Step 1: Manually confirmed FIPS values
manual_fips_fixes_2 = {
    ('IN', 'DeKalb County'): '18033',
    ('IN', 'LaGrange County'): '18087',
    ('IN', 'LaPorte County'): '18091',
    ('AK', 'Anchorage Municipality'): '02020',
    ('AK', 'Juneau City and Borough'): '02110',
    ('AK', 'Petersburg Borough'): '02195',
    ('AK', 'Sitka City and Borough'): '02220',
    ('AK', 'Wrangell City and Borough'): '02275',
    ('AK', 'Yakutat City and Borough'): '02282',
    ('MO', 'Sullivan part'): '29211',  # Closest FIPS: Sullivan County, MO
    ('NM', 'De Baca County'): '35011',
    ('PA', 'McKean County'): '42083'
}


In [23]:
# Step 2: Update FIPS_corrected using manual fixes only where FIPS_corrected is missing
fmr_df['FIPS_corrected'] = fmr_df.apply(
    lambda row: manual_fips_fixes.get((row['State'], row['County']), row['FIPS_corrected']),
    axis=1
)


In [24]:
fmr_df

,FIPS,State,County,fmr_0_2021,fmr_1_2021,fmr_2_2021,fmr_3_2021,fmr_4_2021,fmr_0_2020,fmr_1_2020,...,fmr_1_2022,fmr_2_2022,fmr_3_2022,fmr_4_2022,fmr_0_2024,fmr_1_2024,fmr_2_2024,fmr_3_2024,fmr_4_2024,FIPS_corrected
0,10001,CT,Windham County,816.0,818.0,1036.0,1307.0,1695.0,787.0,811.0,...,854.0,1110.0,1388.0,1875.0,1186.0,1194.0,1569.0,1921.0,2380.0,09015
1,10003,DE,Kent County,878.0,908.0,1066.0,1526.0,1785.0,880.0,893.0,...,856.0,1002.0,1423.0,1647.0,1110.0,1117.0,1368.0,1928.0,2152.0,10001
2,10005,DE,New Castle County,900.0,1040.0,1260.0,1567.0,1796.0,864.0,1013.0,...,1071.0,1298.0,1605.0,1837.0,1303.0,1451.0,1737.0,2100.0,2422.0,10003
3,01001,AL,Autauga County,640.0,766.0,908.0,1148.0,1520.0,583.0,702.0,...,764.0,914.0,1156.0,1494.0,836.0,913.0,1092.0,1383.0,1753.0,01001
4,01003,AL,Baldwin County,718.0,723.0,922.0,1249.0,1584.0,744.0,749.0,...,777.0,1017.0,1348.0,1715.0,1051.0,1056.0,1362.0,1670.0,2114.0,01003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247,09011,CT,New London County,828.0,991.5,1254.5,1709.5,2123.5,826.5,953.5,...,1020.5,1286.0,1742.5,2179.0,1102.5,1277.0,1626.0,2134.5,2740.5,09011
3248,09013,CT,New London County,810.0,976.0,1227.0,1584.0,2027.0,815.0,938.0,...,1006.0,1254.0,1616.0,2102.0,1055.0,1263.0,1556.0,2030.0,2625.0,09011
3249,09013,CT,Tolland County,889.0,1091.0,1347.0,1675.0,1958.0,801.0,993.0,...,1054.0,1302.0,1609.0,1907.0,1121.0,1319.0,1654.0,2000.0,2425.0,09013
3250,09015,CT,Tolland County,889.0,1091.0,1347.0,1675.0,1958.0,801.0,993.0,...,1054.0,1302.0,1609.0,1907.0,1121.0,1319.0,1654.0,2000.0,2425.0,09013


In [25]:
# Show all columns
print(fmr_df.columns.tolist())


['FIPS', 'State', 'County', 'fmr_0_2021', 'fmr_1_2021', 'fmr_2_2021', 'fmr_3_2021', 'fmr_4_2021', 'fmr_0_2020', 'fmr_1_2020', 'fmr_2_2020', 'fmr_3_2020', 'fmr_4_2020', 'fmr_0_2023', 'fmr_1_2023', 'fmr_2_2023', 'fmr_3_2023', 'fmr_4_2023', 'fmr_0_2022', 'fmr_1_2022', 'fmr_2_2022', 'fmr_3_2022', 'fmr_4_2022', 'fmr_0_2024', 'fmr_1_2024', 'fmr_2_2024', 'fmr_3_2024', 'fmr_4_2024', 'FIPS_corrected']


In [26]:
# Step 1: Drop all other FIPS_corrected columns except the last one
fmr_df = fmr_df.drop(columns=[col for col in fmr_df.columns if col.startswith('FIPS_corrected_')])

# Step 2: Rename the surviving one to keep it clear
fmr_df = fmr_df.rename(columns={'FIPS_corrected': 'FIPS_corrected_clean'})

# Step 3 (optional): Create final FIPS column — fallback to original if corrected is missing
fmr_df['FIPS_final'] = fmr_df['FIPS_corrected_clean'].combine_first(fmr_df['FIPS'])


In [27]:
fmr_df

,FIPS,State,County,fmr_0_2021,fmr_1_2021,fmr_2_2021,fmr_3_2021,fmr_4_2021,fmr_0_2020,fmr_1_2020,...,fmr_2_2022,fmr_3_2022,fmr_4_2022,fmr_0_2024,fmr_1_2024,fmr_2_2024,fmr_3_2024,fmr_4_2024,FIPS_corrected_clean,FIPS_final
0,10001,CT,Windham County,816.0,818.0,1036.0,1307.0,1695.0,787.0,811.0,...,1110.0,1388.0,1875.0,1186.0,1194.0,1569.0,1921.0,2380.0,09015,09015
1,10003,DE,Kent County,878.0,908.0,1066.0,1526.0,1785.0,880.0,893.0,...,1002.0,1423.0,1647.0,1110.0,1117.0,1368.0,1928.0,2152.0,10001,10001
2,10005,DE,New Castle County,900.0,1040.0,1260.0,1567.0,1796.0,864.0,1013.0,...,1298.0,1605.0,1837.0,1303.0,1451.0,1737.0,2100.0,2422.0,10003,10003
3,01001,AL,Autauga County,640.0,766.0,908.0,1148.0,1520.0,583.0,702.0,...,914.0,1156.0,1494.0,836.0,913.0,1092.0,1383.0,1753.0,01001,01001
4,01003,AL,Baldwin County,718.0,723.0,922.0,1249.0,1584.0,744.0,749.0,...,1017.0,1348.0,1715.0,1051.0,1056.0,1362.0,1670.0,2114.0,01003,01003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247,09011,CT,New London County,828.0,991.5,1254.5,1709.5,2123.5,826.5,953.5,...,1286.0,1742.5,2179.0,1102.5,1277.0,1626.0,2134.5,2740.5,09011,09011
3248,09013,CT,New London County,810.0,976.0,1227.0,1584.0,2027.0,815.0,938.0,...,1254.0,1616.0,2102.0,1055.0,1263.0,1556.0,2030.0,2625.0,09011,09011
3249,09013,CT,Tolland County,889.0,1091.0,1347.0,1675.0,1958.0,801.0,993.0,...,1302.0,1609.0,1907.0,1121.0,1319.0,1654.0,2000.0,2425.0,09013,09013
3250,09015,CT,Tolland County,889.0,1091.0,1347.0,1675.0,1958.0,801.0,993.0,...,1302.0,1609.0,1907.0,1121.0,1319.0,1654.0,2000.0,2425.0,09013,09013


In [28]:
# Drop both FIPS and corrected version (after FIPS_final has been created)
fmr_df = fmr_df.drop(columns=['FIPS', 'FIPS_corrected_clean'])

# ✅ Check result
print(fmr_df.columns.tolist())


['State', 'County', 'fmr_0_2021', 'fmr_1_2021', 'fmr_2_2021', 'fmr_3_2021', 'fmr_4_2021', 'fmr_0_2020', 'fmr_1_2020', 'fmr_2_2020', 'fmr_3_2020', 'fmr_4_2020', 'fmr_0_2023', 'fmr_1_2023', 'fmr_2_2023', 'fmr_3_2023', 'fmr_4_2023', 'fmr_0_2022', 'fmr_1_2022', 'fmr_2_2022', 'fmr_3_2022', 'fmr_4_2022', 'fmr_0_2024', 'fmr_1_2024', 'fmr_2_2024', 'fmr_3_2024', 'fmr_4_2024', 'FIPS_final']


In [29]:
# Step 1: Drop original FIPS columns
fmr_df = fmr_df.drop(columns=['FIPS', 'FIPS_corrected_clean'], errors='ignore')

# Step 2: Rename final column
fmr_df = fmr_df.rename(columns={'FIPS_final': 'FIPS'})

# Step 3: Reorder — move FIPS to 3rd column
cols = fmr_df.columns.tolist()
fips_index = cols.index('FIPS')
# Remove FIPS and insert at index 2 (3rd position)
cols.insert(2, cols.pop(fips_index))
fmr_df = fmr_df[cols]

# ✅ Preview
print(fmr_df.columns[:5])


Index(['State', 'County', 'FIPS', 'fmr_0_2021', 'fmr_1_2021'], dtype='object')


In [30]:
fmr_df

,State,County,FIPS,fmr_0_2021,fmr_1_2021,fmr_2_2021,fmr_3_2021,fmr_4_2021,fmr_0_2020,fmr_1_2020,...,fmr_0_2022,fmr_1_2022,fmr_2_2022,fmr_3_2022,fmr_4_2022,fmr_0_2024,fmr_1_2024,fmr_2_2024,fmr_3_2024,fmr_4_2024
0,CT,Windham County,09015,816.0,818.0,1036.0,1307.0,1695.0,787.0,811.0,...,848.0,854.0,1110.0,1388.0,1875.0,1186.0,1194.0,1569.0,1921.0,2380.0
1,DE,Kent County,10001,878.0,908.0,1066.0,1526.0,1785.0,880.0,893.0,...,851.0,856.0,1002.0,1423.0,1647.0,1110.0,1117.0,1368.0,1928.0,2152.0
2,DE,New Castle County,10003,900.0,1040.0,1260.0,1567.0,1796.0,864.0,1013.0,...,940.0,1071.0,1298.0,1605.0,1837.0,1303.0,1451.0,1737.0,2100.0,2422.0
3,AL,Autauga County,01001,640.0,766.0,908.0,1148.0,1520.0,583.0,702.0,...,643.0,764.0,914.0,1156.0,1494.0,836.0,913.0,1092.0,1383.0,1753.0
4,AL,Baldwin County,01003,718.0,723.0,922.0,1249.0,1584.0,744.0,749.0,...,772.0,777.0,1017.0,1348.0,1715.0,1051.0,1056.0,1362.0,1670.0,2114.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247,CT,New London County,09011,828.0,991.5,1254.5,1709.5,2123.5,826.5,953.5,...,854.5,1020.5,1286.0,1742.5,2179.0,1102.5,1277.0,1626.0,2134.5,2740.5
3248,CT,New London County,09011,810.0,976.0,1227.0,1584.0,2027.0,815.0,938.0,...,833.0,1006.0,1254.0,1616.0,2102.0,1055.0,1263.0,1556.0,2030.0,2625.0
3249,CT,Tolland County,09013,889.0,1091.0,1347.0,1675.0,1958.0,801.0,993.0,...,865.0,1054.0,1302.0,1609.0,1907.0,1121.0,1319.0,1654.0,2000.0,2425.0
3250,CT,Tolland County,09013,889.0,1091.0,1347.0,1675.0,1958.0,801.0,993.0,...,865.0,1054.0,1302.0,1609.0,1907.0,1121.0,1319.0,1654.0,2000.0,2425.0


In [31]:
# Check full row duplicates
full_duplicates = fmr_df.duplicated()
print(f"❗ Full duplicate rows: {full_duplicates.sum()}")

# Check duplicates based on FIPS, State, and County
key_duplicates = fmr_df.duplicated(subset=['FIPS', 'State', 'County'], keep=False)
print(f"❗ Duplicate rows based on FIPS + State + County: {key_duplicates.sum()}")

# Optional: Show those rows
if key_duplicates.sum() > 0:
    print(fmr_df[key_duplicates].sort_values(by=['FIPS', 'State', 'County']))


❗ Full duplicate rows: 87
❗ Duplicate rows based on FIPS + State + County: 166
     State             County   FIPS  fmr_0_2021  fmr_1_2021  fmr_2_2021  \
3237    CT   Fairfield County  09001      1138.0      1350.0      1725.0   
3238    CT   Fairfield County  09001      1292.0      1621.0      1958.0   
3239    CT    Hartford County  09003       889.0      1091.0      1347.0   
3240    CT    Hartford County  09003       889.0      1091.0      1347.0   
3241    CT  Litchfield County  09005       806.0       950.0      1222.0   
...    ...                ...    ...         ...         ...         ...   
2763    VT     Windsor County  50027       715.0       837.0      1007.0   
2764    VT     Windsor County  50027       715.0       837.0      1007.0   
2765    VT     Windsor County  50027       715.0       837.0      1007.0   
2766    VT     Windsor County  50027       715.0       837.0      1007.0   
2767    VT     Windsor County  50027       715.0       837.0      1007.0   

      fm

In [32]:
# Step 1: Count full duplicates
full_dups = fmr_df.duplicated()
print(f"❗ Fully duplicated rows to drop: {full_dups.sum()}")

# Step 2: Drop them
fmr_df = fmr_df.drop_duplicates()

# ✅ Confirm new shape
print(f"✅ fmr_df shape after dropping full duplicates: {fmr_df.shape}")


❗ Fully duplicated rows to drop: 87
✅ fmr_df shape after dropping full duplicates: (3165, 28)


In [33]:
# Check for duplicate rows based on FIPS, State, and County
key_duplicates = fmr_df.duplicated(subset=['FIPS', 'State', 'County'], keep=False)

# Filter and show
duplicate_rows = fmr_df[key_duplicates].sort_values(by=['FIPS', 'State', 'County'])
print(duplicate_rows)


     State             County   FIPS  fmr_0_2021  fmr_1_2021  fmr_2_2021  \
3237    CT   Fairfield County  09001      1138.0      1350.0      1725.0   
3238    CT   Fairfield County  09001      1292.0      1621.0      1958.0   
3243    CT   Middlesex County  09007       946.5      1123.0      1434.5   
3244    CT   Middlesex County  09007      1004.0      1155.0      1522.0   
3245    CT   New Haven County  09009       939.0      1177.0      1423.0   
3246    CT   New Haven County  09009      1055.0      1181.0      1438.0   
3247    CT  New London County  09011       828.0       991.5      1254.5   
3248    CT  New London County  09011       810.0       976.0      1227.0   
964     ME  Cumberland County  23005       981.5      1079.5      1401.0   
965     ME  Cumberland County  23005      1088.0      1229.0      1592.0   
978     ME   Penobscot County  23019       651.5       742.0       962.0   
979     ME   Penobscot County  23019       595.0       658.0       867.0   
990     ME  

In [34]:
# Step 1: Group by FIPS, State, County and average all other columns
fmr_df = fmr_df.groupby(['FIPS', 'State', 'County'], as_index=False).mean(numeric_only=True)

# ✅ Confirm shape after deduplication
print(f"✅ Shape after resolving duplicates: {fmr_df.shape}")


✅ Shape after resolving duplicates: (3146, 28)


In [35]:
# 🔍 Check for fully duplicated rows
full_dups = fmr_df.duplicated()
print(f"❗ Full duplicate rows: {full_dups.sum()}")

# 🔍 Check for duplicates based on FIPS, State, County
key_dups = fmr_df.duplicated(subset=['FIPS', 'State', 'County'], keep=False)
print(f"❗ Duplicate rows based on FIPS + State + County: {key_dups.sum()}")


❗ Full duplicate rows: 0
❗ Duplicate rows based on FIPS + State + County: 0


In [36]:
print(f"merged_df_1 rows: {merged_df_1.shape[0]}")
print(f"merged_df_2 rows: {merged_df_2.shape[0]}")
print(f"fmr_df rows: {fmr_df.shape[0]}")


merged_df_1 rows: 3198
merged_df_2 rows: 3145
fmr_df rows: 3146


In [37]:
# 🧱 Step 1: Start with merged_df_1 as the base
# We'll merge everything into this DataFrame

# 📦 Step 2: Merge population data (merged_df_2) using FIPS, State, County
df_merged = pd.merge(
    merged_df_1,
    merged_df_2,
    on=['FIPS', 'State', 'County'],
    how='left',  # Keep all rows from merged_df_1, even if there's no match
    suffixes=('', '_pop')
)

# 🏠 Step 3: Merge FMR data (fmr_df) using FIPS, State, County
df_merged = pd.merge(
    df_merged,
    fmr_df,
    on=['FIPS', 'State', 'County'],
    how='left',  # Again, keep all rows from base dataset
    suffixes=('', '_fmr')
)

# ✅ Step 4: Confirm shape and preview
print(f"✅ Final merged dataset shape: {df_merged.shape}")
print(df_merged[['FIPS', 'State', 'County']].head())


✅ Final merged dataset shape: (3198, 322)
    FIPS State          County
0  00000    US   United States
1  01000    AL         Alabama
2  01001    AL  Autauga County
3  01003    AL  Baldwin County
4  01005    AL  Barbour County


In [38]:
# Step 1: Check missing values
missing_summary = df_merged.isna().sum()
missing_summary = missing_summary[missing_summary > 0]

# Step 2: Identify unmatched rows from merged_df_2 and fmr_df
merged_2_columns = [col for col in merged_df_2.columns if col not in ['FIPS', 'State', 'County']]
fmr_columns = [col for col in fmr_df.columns if col not in ['FIPS', 'State', 'County']]

missing_merged_2 = df_merged[merged_2_columns].isna().all(axis=1).sum()
missing_fmr = df_merged[fmr_columns].isna().all(axis=1).sum()

# Step 3: Check for key-based duplicates
dup_count = df_merged.duplicated(subset=['FIPS', 'State', 'County']).sum()

# Step 4: Print summary
summary = {
    "Total rows": df_merged.shape[0],
    "Rows missing any value": int(df_merged.isna().any(axis=1).sum()),
    "Rows missing all merged_df_2 columns": int(missing_merged_2),
    "Rows missing all fmr_df columns": int(missing_fmr),
    "Duplicate rows (FIPS + State + County)": int(dup_count),
    "Columns with missing values": missing_summary.to_dict()
}
print(summary)


{'Total rows': 3198, 'Rows missing any value': 3198, 'Rows missing all merged_df_2 columns': 75, 'Rows missing all fmr_df columns': 64, 'Duplicate rows (FIPS + State + County)': 0, 'Columns with missing values': {'air_pollution__air_pollution_2020': 1, 'air_pollution__air_pollution_2021': 1, 'air_pollution__air_pollution_2022': 1, 'air_pollution__air_pollution_2023': 1, 'air_pollution__air_pollution_2024': 1, 'air_pollution__air_pollution_mean': 1, 'children_lunch__children_lunch_2020': 118, 'children_lunch__children_lunch_2021': 118, 'children_lunch__children_lunch_2022': 22, 'children_lunch__children_lunch_2023': 586, 'children_lunch__children_lunch_2024': 549, 'children_lunch__children_lunch_mean': 22, 'children_poverty__children_poverty_2020': 1, 'children_poverty__children_poverty_2021': 1, 'children_poverty__children_poverty_2022': 1, 'children_poverty__children_poverty_2023': 1, 'children_poverty__children_poverty_2024': 1, 'children_poverty__children_poverty_mean': 1, 'dentists

In [39]:
missing_rows = df_merged[df_merged.isna().any(axis=1)]

In [40]:
df_merged['missing_fmr'] = df_merged[fmr_columns].isna().all(axis=1)
df_merged['missing_demo'] = df_merged[merged_2_columns].isna().all(axis=1)


In [41]:
# Set your export folder path
export_folder = '/Users/emilsafarov/Library/CloudStorage/OneDrive-Personal/CF/A6/1_sourceing_open_data/US project/Data/Output'
# Export as CSV
df_merged.to_csv(f"{export_folder}/final_merged_dataset.csv", index=False)